### Deep Dive into LangChain - LLMs, Prompt Templates, Caching, Streaming, Chains

#### Python-dotenv: used to securely retrieve the environment variables

In [1]:
import os
from dotenv import load_dotenv, find_dotenv

# loading the .env file
load_dotenv(find_dotenv(), override=True)

# Getting the  OpenAI API Keys from the .env file
openAPIKey = os.environ.get('OPENAI_API_KEY')

# Getting the ElevenLabs API Key
elevenLabsKey = os.environ.get("ELEVEN_API_KEY")

### How to Invoke different Chat Models: GPT-3.5 Turbo and GPT-4

##### 1. Using ChatOpenAI object

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(temperature=0.1, model='gpt-4o', api_key=openAPIKey)

# Invoking the LLM model
output = llm.invoke('Explain quantum mechanics in one sentence.')

# Invoking the GPT-4
#output = llm.invoke('Explain quantum mechanics in one sentence.', model='gpt-4', temperature='0.1')

print(output.content)

Quantum mechanics is the branch of physics that deals with the behavior of particles at atomic and subatomic scales, where phenomena such as superposition, entanglement, and quantization of energy levels are fundamental.


##### 2. Using ChatCompletion

In [ ]:
# Using Chat Completions API Messages: System, Assistant and Human
from langchain.schema import(SystemMessage, AIMessage, HumanMessage)

messages = [
    SystemMessage(content='You are a physicist and respond only in German.'),
    HumanMessage(content='Explain quantum mechanics in one sentence.')
]

output = llm.invoke(messages)
print(output.content)

### Caching LLM Responses

##### 1. In-Memory Cache

In [ ]:
from langchain.globals import set_llm_cache
from langchain_openai import OpenAI
from langchain.cache import InMemoryCache

# Setting the in-memory cache
set_llm_cache(InMemoryCache())

# Defining the OpenAI object
llm = OpenAI(model_name='gpt-3.5-turbo-instruct')   # It's a slower model, for demo purposes only

##### Making the first LLM request

In [ ]:
%%time
prompt = 'Tell a me a joke that a toddler can understand.'
llm.invoke(prompt)

##### Making the Second LLM request

In [ ]:
%%time
llm.invoke(prompt)

##### 2. SQLite Caching

In [ ]:
# Setting the SQLite cache
from langchain.cache import SQLiteCache

# Creating a local db for storing cache information
set_llm_cache(SQLiteCache(database_path=".langchain.db"))

In [ ]:
%%time
# First request (not in cache, takes longer)
llm.invoke("Tell me a joke")

In [ ]:
%%time
# Second request (cached, faster)
llm.invoke("Tell me a joke")

### LLM Streaming

In [ ]:
from langchain_openai import ChatOpenAI

# Defining ChatOpenAI object
llm = ChatOpenAI()
prompt = 'Write a rock song about the Moon and a Raven.'

# Making the API call
print(llm.invoke(prompt).content)

In [ ]:
# Let's Implement Streaming
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

### PromptTemplates - used for tasks that involves 'Text Generation' or 'Completion'

In [ ]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

# Define a template for the prompt
template = '''You are an experienced virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Create a PromptTemplate object from the template
prompt_template = PromptTemplate.from_template(template=template)

# Fill in the variable: 'virus' and 'language'
prompt = prompt_template.format(virus='Covid', language='English')

# Returns the generated prompt
prompt

##### Making the API call

In [ ]:
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

# Native Response
#output = llm.invoke(prompt)
#print(output.content)

# Streaming response
for chunk in llm.stream(prompt):
    print(chunk.content, end='', flush=True)

### ChatPromptTemplates -  used for tasks that involves 'Engaging in Conversation'

In [ ]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

# Create a chat template with system and human messages
chat_template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content='You respond only in the JSON format.'),
        HumanMessagePromptTemplate.from_template('Top {n} countries in {area} by population.')
    ]
)

# Fill in the specific values for n and area
messages = chat_template.format_messages(n='5', area='World')

# Outputs the formatted chat messages
print(messages)

In [ ]:
# Defining ChatOpenAI object
llm = ChatOpenAI()

# Native Response
#output = llm.invoke(messages)
#print(output.content)

# Streaming Response
for chunks in llm.stream(messages):
    print(chunks.content, end="", flush=True)

### Simple Chains

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain

# Defining ChatOpenAI model
llm = ChatOpenAI()

# Defining Template
template = '''You are an experience virologist.
Write a few sentences about the following virus "{virus}" in {language}.'''

# Setting the prompt template
prompt_template = PromptTemplate.from_template(template=template)

# Declaring a LLM Chain
chain = LLMChain(llm=llm, prompt=prompt_template, verbose=True)

# Getting the Output
output = chain.invoke({'virus': 'HSV', 'language': 'Spanish'})


In [ ]:
print(output)

In [ ]:
template = 'What is the capital of {country}?. List the top 3 places to visit in that city. Use bullet points'
prompt_template = PromptTemplate.from_template(template=template)

# Initialize an LLMChain with the ChatOpenAI model and the prompt template
chain = LLMChain(
    llm=llm,
    prompt=prompt_template,
    verbose=True
)

country = input('Enter Country: ')

# Invoke the chain with specific virus and language values
output = chain.invoke({'country': country})
print(output['text'])

### Simple Sequential Chains: First chain's Output is used as Second chain's Input

`Each chain has 1 input and 1 output`

In [ ]:
from langchain_openai import ChatOpenAI
from langchain import PromptTemplate
from langchain.chains import LLMChain, SimpleSequentialChain

# Initialize the first ChatOpenAI model (gpt-3.5-turbo) with specific temperature
llm1 = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0.5)

# Define the first prompt template
prompt_template1 = PromptTemplate.from_template(
    template='You are an experienced scientist and Python programmer. Write a function that implements the concept of {concept}.'
)

# Create an LLMChain using the first model and the prompt template
chain1 = LLMChain(llm=llm1, prompt=prompt_template1)

# Initialize the second ChatOpenAI model (gpt-4-turbo) with specific temperature
llm2 = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=1.2)

# Define the second prompt template
prompt_template2 = PromptTemplate.from_template(
    template='Given the Python function {function}, describe it as detailed as possible.'
)
# Create another LLMChain using the second model and the prompt template
chain2 = LLMChain(llm=llm2, prompt=prompt_template2)

# Combine both chains into a SimpleSequentialChain
overall_chain = SimpleSequentialChain(chains=[chain1, chain2], verbose=True)

# Invoke the overall chain while specifying only the 'Input Variable' for the 'First Chain' 
output = overall_chain.invoke('linear regression')

# Print the results
print(output['output'])

### Run Python code using LangChain: PythonREPL

In [ ]:
from langchain_experimental.utilities import PythonREPL

# Create PythonREPL agent object
python_repl = PythonREPL()

# Run the Python code
python_repl.run('print([n for n in range(1, 100) if n % 13 == 0])')

### LangChain Agents

1. The core idea of agents is to use large language models to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

2. Agents help LLMs to run code, do calculations, search web or run SQL queries

In [ ]:
from langchain_experimental.agents.agent_toolkits import create_python_agent    # Special type of LangChain agent to interact with Python code
from langchain_experimental.tools.python.tool import PythonREPLTool # Tool to provide Python environment within LangChain allowing LangChain agent to execute py code
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model with gpt-4-turbo and a temperature of 0
llm = ChatOpenAI(model='gpt-4-turbo-preview', temperature=0)

# Create a Python agent using the ChatOpenAI model and a PythonREPLTool
# Tools are functions that allow agents to interact with the outside world
agent_executor = create_python_agent(llm=llm, tool=PythonREPLTool(), verbose=True)

# Set the Prompt
prompt = 'Calculate the square root of the factorial of 12 and display it with 4 decimal points'

# Invoke the agent
agent_executor.invoke(prompt)

### LangChain Tools: DuckDuckGo and Wikipedia

##### 1. DuckDuckGoSearchRun: To Query the internet

In [ ]:
from langchain.tools import DuckDuckGoSearchRun

# Declare DuckDuckGoSearch Object
search = DuckDuckGoSearchRun()

# Get more info about DuckDuckGoSearchRun()
print(search.description)

# Make the call
output = search.invoke('Where was Freddie Mercury born?')

print(output)

##### 2. DuckDuckGoSearchResults: To get additional information and links to the search

In [ ]:
from langchain.tools import DuckDuckGoSearchResults

# Define Search Result Object
search = DuckDuckGoSearchResults()

# Invoke the call
output = search.run('Freddie Mercury and Queen.')

# Print the Output
print(output)

##### 3. DuckDuckGoSearchAPIWrapper

In [ ]:
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper

# Define Wrapper Object
wrapper = DuckDuckGoSearchAPIWrapper(region='in', max_results=3, safesearch='moderate')

# Define Search Result Object
search = DuckDuckGoSearchResults(api_wrapper=wrapper, source='news')

# Invoke the call
output = search.run('India')

# Print the result
print(output)

##### Wikipedia

In [ ]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

# Defining Wiki API Wrapper
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=10000)

# Wiki QueryRun object
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

# Get the output
output = wiki.invoke({'query': 'Google Gemini'})

# Print the Output
print(output)

### Important: Creating a ReACT Agent

##### What is a ReAct Agent?

* ReAct Agent in LangChain: is a system, based on ReAct methodology (Reasoning + Action), that uses LLM model to

    * Analyze user's query, and
    * Decide which actions to take, in which order, and which tools to use (Python REPL, DuckDuckGo, Wikipedia) to get the info requested in the user query

<b>Example:</b>

* <b>User Query:</b> The user asks, “what’s the weather in New York?”

* <b>Thought:</b> The LLM, thinks about what to do next. It decides that it needs to use a 'Weather Tool; to get the current weather in New York.

* <b>Act:</b> The agent uses the weather tool to fetch the weather information. This is like the agent “doing” the action it thought of.

* <b>Observe:</b> The agent observes the result from the weather tool. Let’s say the tool returns that it’s currently sunny and 75 degrees Fahrenheit in New York.

* <b>Thought:</b> The LLM processes this information and decides that it has enough information to answer the user’s question.

* <b>Final Answer:</b> The agent responds to the user, “The current weather in New York is sunny and 75 degrees Fahrenheit.

* So, in simple terms, a ReAct agent uses an LLM to 'Think, Act, and Observe' in a loop until it completes a task

* The user query guides what task the agent needs to complete, and the available tools help the agent gather the information it needs to complete the task

`ReAct Benefits: help overcome Hallucinations and Error Propagations`

In [ ]:
from langchain.prompts import PromptTemplate

# LAngChain Hub: is a place for sharing and discovering resources related to LangChain
from langchain import hub

from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent

# Imports for Tools
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_community.tools import ElevenLabsText2SpeechTool

# Import for ChatOpenAI
from langchain_openai import ChatOpenAI

# Initialize the ChatOpenAI model (gpt-4-turbo-preview) with a temperature of 0. Utilize gpt-3.5-turbo if you use the free plan
llm = ChatOpenAI(model_name='gpt-4-turbo-preview', temperature=0)

# Declare a 'User-defined' template for answering questions
template = '''
Answer the following questions in English as best you can.
Questions: {q}
'''

# Create a PromptTemplate object to use the 'user-defined' template
prompt_template = PromptTemplate.from_template(template)


# Pull the 'ReAct Prompt Template' from the hub
prompt = hub.pull('hwchase17/react')

# displaying information about the react prompt
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)


# Create tools for the agent

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name='Python REPL',
    func=python_repl.run,   #Syntax: <tool>.run
    description='Useful when you need to use Python to answer a question. You should input Python code.'
)

# 2. Wikipedia Tool (for searching Wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on Wikipedia.'
)

# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool can\'t provide.'
)

# 4. ElevenLabs Tool (for Text to Speech)
tts = ElevenLabsText2SpeechTool()
tts_tool = Tool(
    name='ElevenLabs Tool',
    func=tts.run,
    description="Useful for converting Text to Speech"
)

# Combine the tools into a list
tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool, tts_tool]

# Create a 'ReAct Agent' with the ChatOpenAI model, tools, and prompt
agent = create_react_agent(llm, tools, prompt)

# Initialize the AgentExecutor
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=10
)

#### 1. Solving a Python Problem

In [ ]:
question = 'Generate the first 20 numbers in the Fibonacci series.'
output = agent_executor.invoke({'input': prompt_template.format(q=question)})
print(output['output'])

#### 2. Asking a Question

In [ ]:
question = 'Tell me about Napoleon Bonaparte early life'
output = agent_executor.invoke({'input': prompt_template.format(q=question)})
print(output['output'])

#### 3. Text to Speech Conversion

In [ ]:
question = "Convert the following text into speech- Hello World! The king has arrived"
output = agent_executor.invoke({'input': prompt_template.format(q=question)})

### ElevenLabs Standalone Implementation of 'Text to Speech'

In [ ]:
from elevenlabs import play
from elevenlabs.client import ElevenLabs

# Initialize ElevenLabs Object
client = ElevenLabs(api_key=elevenLabsKey)

# Get all the possible voices
outputVoices = client.voices.get_all()

audio = client.generate(
  text="Hi, this is a sample Text to Voice Conversion",
  voice=outputVoices.voices[2],
  model="eleven_multilingual_v2"
)

# To see a list of available voices
outputVoices.voices

# Play the Audio
play(audio)